### R1_P2_P1

In [ ]:
# Solution

### R1_P2_P2

In [ ]:
# Solution

### R1_P2_P3

In [ ]:
# Solution

### R1_P2_P4

In [ ]:
# Solution

### R1_P2_P5

In [ ]:
# Solution

### R1_P2_P6

In [ ]:
# Solution

### R1_P2_P9

In [ ]:
# Solution

### R1_P2_P10

In [ ]:
# Solution

### R1_P2_P11

In [ ]:
# Solution

### R1_P2_P12

In [ ]:
# Solution

### R1_P2_P13

In [ ]:
# Solution

### R1_P3_P1

In [ ]:
# Solution

### R1_P3_P2

In [ ]:
# Solution

### R1_P3_P3

In [ ]:
# Solution

### R1_P3_P4

In [ ]:
# Solution

### R1_P3_P5

In [ ]:
# Solution

### R1_P3_P6

In [ ]:
# Solution

### R1_P3_P7

In [ ]:
# Solution

### R1_P3_P8

In [ ]:
# Solution

### R1_P3_P9

In [ ]:
# Solution

### R1_P3_P16

In [ ]:
# Solution